In [1]:
cd("../")

In [15]:
using CSV
using DataFrames
using LinearAlgebra
using Statistics
using Optim

## P-MEDM Problem and Data

- Census microdata (PUMS) with $n$ = 15 respondents, $N$ = 1286 people.
- Public-Use Microdata Area (PUMA) consisting of 10 block groups.
- **PROBLEM**: PUMS provides no information on the block groups in which each "person" (sample weight unit) in the PUMS resides. 

**Solution: Iterative Proportional Fitting (IPF).** Use census data published for two spatial scales, the target units (block groups) and an upper level (tracts), known as _constraints_ to make a "guess" at where people described by each response type belong. This is done by comparing the published constraints to synthetic constraints reconstructed from each "guess" at the probabilities of people's locations given by P-MEDM. The synthetic constraints are updated iteratively until a "best guess" is reached. 

But...**another problem!** Our census data comes from the American Community Survey (ACS). ACS data is updated annually but is inherently uncertain because it is based on a sample vs. a complete count of the population (i.e., the Decennial Census). 

**Penalized Maximum-Entropy Dasymetric Modeling (P-MEDM)** is an IPF technique specialized to deal with uncertain census data like the ACS (Leyk et al xx; Nagle et al 2014). Besides taking errors between the published and synthetic constraints into account, P-MEDM also accounts for _error variances_. The more "in the ballpark" of the published constraints and their variances the synthetic constraints are, the better the solution. Conversely, if a synthetic constraint is out of the range of probable values (i.e., 90% margin of error) of a published constraint, the more _penalized_ the solution will be.  

In [3]:
## read in data
constraints_ind = CSV.read("data/toy_constraints_ind.csv");
constraints_bg = CSV.read("data/toy_constraints_bg.csv");
constraints_trt = CSV.read("data/toy_constraints_trt.csv");

,GEOID,POP,CONST1,CONST2,CONST3,POPs,CONST1s,CONST2s,CONST3s
,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64
1,10,344,152,101,175,9.43398,7.81025,12.3288,11.1803
2,20,260,84,106,107,4.24264,7.61577,8.24621,7.81025
3,30,682,200,242,293,8.3666,12.49,14.3527,10.0


In [4]:
## build geo lookup
bg_id = string.(collect(constraints_bg[!,1]));
trt_id = [s[1:2] for s in bg_id];
geo_lookup = DataFrame(bg = bg_id, trt = trt_id)

,bg,trt
,String,String
1,101,10
2,102,10
3,103,10
4,201,20
5,202,20
6,301,30
7,302,30
8,303,30
9,304,30


## Setup

In [6]:
## PUMS response ids
serial = collect(constraints_ind.SERIAL);

In [8]:
## PUMS sample weights
wt = collect(constraints_ind.PERWT);

In [9]:
## population and sample size
N = sum(constraints_bg.POP);
n = nrow(constraints_ind);

In [10]:
## Individual-level (PUMS) constraints
excl = ["SERIAL", "PERWT"];
constraint_cols = [i ∉ excl for i in names(constraints_ind)];
pX = constraints_ind[!,constraint_cols];
pX = convert(Matrix, pX);

In [11]:
## geographic constraints
est_cols_bg = [!endswith(i, 's') && i != "GEOID" for i in names(constraints_bg)]
est_cols_trt = [!endswith(i, 's') && i != "GEOID" for i in names(constraints_trt)]
Y1 = convert(Matrix, constraints_trt[!,est_cols_trt])
Y2 = convert(Matrix, constraints_bg[!,est_cols_bg]);

In [12]:
## error variances
se_cols = [endswith(i, 's') for i in names(constraints_bg)];
se_cols = names(constraints_bg)[se_cols];
V1 = map(x -> x^2, convert(Matrix, constraints_trt[!,se_cols]));
V2 = map(x -> x^2, convert(Matrix, constraints_bg[!,se_cols]));

In [13]:
## Geographic crosswalk
A1 = [];

for G in unique(geo_lookup.trt)

    blah = zeros(Int8, 1, nrow(constraints_bg))

    isG = [occursin(G, g) for g in collect(geo_lookup.bg)]
    for i in findall(isG)
        blah[i] = 1
    end
    append!(A1, blah)

end

A1 = reshape(A1, nrow(constraints_bg), nrow(constraints_trt));
A1 = transpose(A1)

3×10 LinearAlgebra.Transpose{Any,Array{Any,2}}:
 1  1  1  0  0  0  0  0  0  0
 0  0  0  1  1  0  0  0  0  0
 0  0  0  0  0  1  1  1  1  1

In [16]:
## Target unit identity matrix
A2 = Matrix(I, nrow(constraints_bg), nrow(constraints_bg))

10×10 Array{Bool,2}:
 1  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1

In [18]:
## Solution space (X-matrix)
X1 = kron(transpose(pX), A1);
X2 = kron(transpose(pX), A2);
X = transpose(vcat(X1, X2));

In [20]:
## Design weights
q = repeat(wt, size(A1)[2]);
q = reshape(q, n, size(A1)[2]);
q = q / sum(q);
q = vec(q');

In [21]:
## Vectorize geo. constraints (Y) and normalize
Y_vec = vcat(vec(Y1), vec(Y2)) / N; 

In [22]:
## Vectorize variances and normalize
V_vec = vcat(vec(V1), vec(V2)) * (n / N^2);

In [24]:
## Diagonal matrix of variances
sV = Diagonal(V_vec);

## Functions

In [26]:
## Compute the P_MEDM probabilities from q, X, λ
compute_allocation = function(q, X, λ)

    a0 = exp.(-X * λ)

    a = a0 .* q;

    b = q' * a0

    a/b

end;

In [27]:
## Primal Function
penalized_entropy = function(w, d, n, N, v)

    e = d - w

    penalty = (e^2 / (2. * v))

    ent = ((n / N) * (w / d) * log((w/d)))

    pe = (-1. * ent) - penalty

    return pe

end;

In [29]:
## Objective Function
neg_pe = function(λ)

    phat = compute_allocation(q, X, λ)
    phat = reshape(phat, size(A2)[1], size(pX)[1])'

    Yhat2 = (N * phat)' * pX

    phat_trt = (phat * N) * A1'
    Yhat1 = phat_trt' * pX

    Yhat = vcat(vec(Yhat1), vec(Yhat2))

    Ype = DataFrame(Y = Y_vec * N, Yhat = Yhat, V = V_vec * (N^2/n))

    pe = penalized_entropy.(Ype.Y, Ype.Yhat, n, N, Ype.V)

    -1. * mean(pe)

end;

## Optimization

In [32]:
opt = optimize(neg_pe, zeros(length(Y_vec)), BFGS(), autodiff = :forward,
            Optim.Options(iterations = 200))

 * Status: success

 * Candidate solution
    Minimizer: [5.36e-01, -3.61e-02, -5.00e-01,  ...]
    Minimum:   -1.553683e-06

 * Found with
    Algorithm:     BFGS
    Initial Point: [0.00e+00, 0.00e+00, 0.00e+00,  ...]

 * Convergence measures
    |x - x'|               = 1.48e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 5.33e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.83e-16 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-10 ≰ 0.0e+00
    |g(x)|                 = 5.04e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    99
    f(x) calls:    281
    ∇f(x) calls:   281


In [34]:
## final coefficients (lambdas/λ)
λ = Optim.minimizer(opt);

In [35]:
## inspect results
phat = compute_allocation(q, X, λ);
phat = reshape(phat, size(A2)[1], size(pX)[1])';

Yhat2 = (N * phat)' * pX;

phat_trt = (phat * N) * A1';
Yhat1 = phat_trt' * pX;

Yhat = vcat(vec(Yhat1), vec(Yhat2));

Ype = DataFrame(Y = Y_vec * N, Yhat = Yhat, V = V_vec * (N^2/n));

#90% MOEs
Ype.MOE_lower = Ype.Y - (sqrt.(Ype.V) * 1.645);
Ype.MOE_upper = Ype.Y + (sqrt.(Ype.V) * 1.645);

# Proportion of contstraints falling outside 90% MOE
sum((Ype.Yhat .< Ype.MOE_lower) + (Ype.Yhat .> Ype.MOE_upper) .>= 1) / nrow(Ype)

0.0